In [42]:
import sys
import os
import numpy as np 
import re
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess

In [43]:
epochs = 80
trainable = True

In [44]:
from keras_contrib.layers import CRF


In [45]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [46]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [47]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [48]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [49]:
print(train_sentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [50]:
word2vec_words=[]

with open('/home/jindal/notebooks/embeddings/embeddings/model_word2vec.embeddings','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
        word2vec_words.append(word)

In [51]:
print(word2vec_words[0])

.


In [52]:
print(train_sentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [53]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [54]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [55]:
print(word2vec_words[:10])

['.', ',', 'der', 'und', '####', 'die', 'in', '"', '-', '##']


In [56]:
print(len(word2vec_words))

373860


In [57]:
'1' in word2vec_words

False

In [59]:
test_sentences_iv=[]
test_sentences_oov=[]

total_no=0
dot_no=0
for sentence in testSentences:
#     print()
    flag=False
    for word in sentence:
        token=word[0]
        token = re.sub('\d','#',token)
#         print(token)
        if token not in word2vec_words:
#             print('no: %s' %token)
            flag=True
            break
#         else:
#             print('yes: %s' %token)
    if flag:
        test_sentences_oov.append(sentence)
    else:
        test_sentences_iv.append(sentence)

In [60]:
print(len(test_sentences_iv))
print(len(test_sentences_oov))
print(len(test_sentences))

2037
3062
5099


In [61]:
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
#             words[token.lower()] = True
#             words[token] = True
            words[re.sub('\d','#',token)] = True

In [62]:
print(labelSet)

{'I-LOCpart', 'I-ORG', 'I-PERpart', 'I-PER', 'B-LOCderiv', 'B-ORGpart', 'B-OTH', 'I-ORGderiv', 'B-LOCpart', 'I-LOC', 'B-PER', 'I-ORGpart', 'I-OTHderiv', 'I-LOCderiv', 'B-PERpart', 'O', 'I-PERderiv', 'B-ORGderiv', 'B-ORG', 'B-LOC', 'B-OTHderiv', 'I-OTH', 'I-OTHpart', 'B-OTHpart', 'B-PERderiv'}


In [63]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [64]:
print(label2Idx)

{'I-LOCpart': 0, 'I-ORGderiv': 7, 'I-OTHpart': 22, 'B-LOCpart': 8, 'B-PERpart': 14, 'I-ORG': 1, 'B-LOCderiv': 4, 'I-PERderiv': 16, 'B-ORGderiv': 17, 'B-ORGpart': 5, 'B-OTH': 6, 'B-ORG': 18, 'B-LOC': 19, 'B-OTHderiv': 20, 'I-PER': 3, 'B-PER': 10, 'I-LOC': 9, 'I-OTHderiv': 12, 'B-OTHpart': 23, 'I-LOCderiv': 13, 'I-OTH': 21, 'O': 15, 'I-ORGpart': 11, 'I-PERpart': 2, 'B-PERderiv': 24}


In [65]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [25]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words.txt','wb')
f.write(string_words.encode())
f.close()

In [66]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'PADDING_TOKEN': 7, 'numeric': 0, 'contains_digit': 6, 'allLower': 1, 'other': 4, 'initialUpper': 3, 'allUpper': 2, 'mainly_numeric': 5}


In [67]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vec.embeddings", 'rb')

for line in fEmbeddings:
    split = line.strip().split()
    word = split[0].decode()
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(300) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, 300)
        wordEmbeddings.append(vector)

    if word in words:
        
#         for num in split[1:]:
#             print(float(num))
        vector = np.array([float(num.decode()) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [69]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[2]))

60615
300


In [70]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{',': 0, ')': 1, 'ę': 2, 'ő': 4, '_': 5, '§': 6, 'Ş': 10, 'ă': 8, 'ć': 11, 'š': 12, 'È': 13, ']': 237, 'Ġ': 14, 'Æ': 15, '佐': 16, 'п': 17, 'c': 18, 'q': 19, 'u': 20, 'М': 23, 'р': 216, 'φ': 24, '7': 3, 'µ': 26, 'н': 60, 'õ': 28, 'ı': 29, 'ы': 248, '章': 30, 'ñ': 168, 'ن': 32, 'м': 34, 'Þ': 166, '\xad': 35, '}': 36, '台': 152, '별': 38, '`': 279, '−': 39, '算': 7, 'ž': 41, 'オ': 42, '妃': 43, 'Š': 44, 'ß': 45, 'O': 47, '›': 51, 'w': 49, 'κ': 52, '†': 53, 'ą': 55, 'ż': 59, '守': 58, '\x99': 178, 'η': 62, 'π': 61, 'α': 9, 'j': 63, '€': 64, 'ế': 284, 'S': 65, 'Т': 66, 'U': 67, 'E': 68, '±': 283, 'ъ': 71, 'ř': 69, 'W': 215, 'k': 72, 'ю': 73, '°': 118, 'H': 224, '\x96': 74, 'т': 76, 'ã': 77, '학': 79, 'è': 78, '<': 185, 'أ': 80, '—': 81, 'á': 119, 'ν': 83, '3': 227, '“': 84, 'l': 181, 'n': 86, '-': 88, 'ό': 235, 'υ': 95, 'm': 90, 'M': 91, 'i': 92, 'p': 93, '樓': 98, '→': 222, 'K': 96, '=': 232, 'ж': 99, 'v': 100, 'ē': 101, 'ʻ': 238, '?': 102, 'r': 104, 'д': 105, '×': 107, 'İ': 219, '«': 108, 'å': 109

In [71]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [72]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_iv= padding(createMatrices(test_sentences_iv, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_oov = padding(createMatrices(test_sentences_oov, word2Idx, label2Idx, case2Idx, char2Idx))

In [73]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))

[[56272, 2573, 25, 9, 21310, 9, 79, 6779, 3, 2272, 275, 9, 8, 47, 1053, 10800, 3, 7, 872, 234, 28, 16151, 35, 9, 2], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[ 65,  18, 179, ...,   0,   0,   0],
       [214, 180, 298, ...,   0,   0,   0],
       [320, 200,  90, ...,   0,   0,   0],
       ...,
       [ 49, 180, 104, ...,   0,   0,   0],
       [260,   0,   0, ...,   0,   0,   0],
       [290,   0,   0, ...,   0,   0,   0]], dtype=int32), [10, 15, 15, 15, 18, 15, 15, 15, 15, 10, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]]
25
[[ 65  18 179 ...   0   0   0]
 [214 180 298 ...   0   0   0]
 [320 200  90 ...   0   0   0]
 ...
 [ 49 180 104 ...   0   0   0]
 [260   0   0 ...   0   0   0]
 [290   0   0 ...   0   0   0]]


In [74]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [75]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [76]:
print(test_sentences_iv[0])

[['1951', ['1', '9', '5', '1'], 'O'], ['bis', ['b', 'i', 's'], 'O'], ['1953', ['1', '9', '5', '3'], 'O'], ['wurde', ['w', 'u', 'r', 'd', 'e'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['nördliche', ['n', 'ö', 'r', 'd', 'l', 'i', 'c', 'h', 'e'], 'O'], ['Teil', ['T', 'e', 'i', 'l'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['Jugendburg', ['J', 'u', 'g', 'e', 'n', 'd', 'b', 'u', 'r', 'g'], 'O'], ['des', ['d', 'e', 's'], 'O'], ['Kolpingwerkes', ['K', 'o', 'l', 'p', 'i', 'n', 'g', 'w', 'e', 'r', 'k', 'e', 's'], 'B-OTH'], ['gebaut', ['g', 'e', 'b', 'a', 'u', 't'], 'O'], ['.', ['.'], 'O']]


In [77]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10496       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_11 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_14 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [78]:
for x in range(1):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
#             predLabels = np.concatenate(predLabels).ravel()
#             correctLabels = np.concatenate(correctLabels).ravel()
#             f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
            print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
#             print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_oov_word2vec.h5')
                maxf1 = f1_dev

    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    

Epoch 0/80
55/51 [================================] - 19s 342ms/step
Epoch 1/80
55/51 [================================] - 12s 219ms/step
Epoch 2/80
55/51 [================================] - 12s 224ms/step
Epoch 3/80
55/51 [================================] - 12s 209ms/step
Epoch 4/80
55/51 [================================] - 12s 222ms/step
Epoch 5/80
55/51 [================================] - 12s 213ms/step
Epoch 6/80
55/51 [================================] - 12s 227ms/step
Epoch 7/80
55/51 [================================] - 12s 224ms/step
Epoch 8/80
55/51 [================================] - 12s 216ms/step
Epoch 9/80
55/51 [================================] - 12s 226ms/step
Epoch 10/80
55/51 [================================] - 12s 210ms/step
Epoch 11/80
55/51 [================================] - 12s 219ms/step
Epoch 12/80
55/51 [================================] - 12s 218ms/step
Epoch 13/80
55/51 [================================] - 12s 214ms/step
Epoch 14/80
55/51 [===========

2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.833, Rec: 0.786, F1: 0.809
Epoch 59/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.829, Rec: 0.791, F1: 0.810
Epoch 60/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.805, Rec: 0.795, F1: 0.800
Epoch 61/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.813, Rec: 0.782, F1: 0.797
Epoch 62/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.827, Rec: 0.792, F1: 0.809
Epoch 63/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.833, Rec: 0.788, F1: 0.810
Epoch 64/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.819, Rec: 0.794, F1: 0.806
Epoch 65/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.825, Rec: 0.796, F1: 0.810
Epoch 66/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.828, Rec: 0.788, F1: 0.807
Epoch 67/80
2198/2199 [========

In [80]:
model.load_weights('german_ner_oov_word2vec.h5')
test_batch_oov, y = createBatches(test_set_oov)
predLabels, correctLabels = tag_dataset(test_batch_oov)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_oov_word2vec'+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_oov):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_oov_word2vec'+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

3061/3062 [============================>.] - ETA: 0sDev-Data: Prec: 0.802, Rec: 0.752, F1: 0.777
******************************************************************************


In [81]:
model.load_weights('german_ner_oov_word2vec.h5')
test_batch_iv , x =  createBatches(test_set_iv)
predLabels, correctLabels = tag_dataset(test_batch_iv)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_iv_word2vec'+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_iv):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_iv_word2vec'+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

2036/2037 [============================>.] - ETA: 0sDev-Data: Prec: 0.876, Rec: 0.857, F1: 0.866
******************************************************************************
